In [8]:
#!/usr/bin/python

import re
import random
import sys
import copy

# - variables
round_brackets_stack = []
curly_brackets_stack = []
comments = ""

# - data types
dataTypes = ['int', 'char' , 'boolean', 'String','long','void']

# - read java code
input = open(r"C:\Backup\java files\Try_Catch.java",'r')
allJavaCode = input.read()

# - remove comments
for i in range(0,len(allJavaCode)):
    if i <len(allJavaCode)-1   and allJavaCode[i] == "/" and allJavaCode[i+1] == "/":
        while allJavaCode[i] != "\n":
            comments =  comments + allJavaCode[i] 
            i = i +1

        allJavaCode =  allJavaCode.replace(comments,'')
    comments = ""

#print allJavaCode

# - check for the opening and closing brackets
for i in range(0,len(allJavaCode)):
    if allJavaCode[i] == '(':
        round_brackets_stack.append("(")
    if allJavaCode[i] == ')':
        try: 
            round_brackets_stack.pop()
        except:
            print("missing a round bracket ")
            sys.exit()

    if allJavaCode[i] == '{':
        curly_brackets_stack.append("{")
    if allJavaCode[i] == '}':
        try:
            curly_brackets_stack.pop()
        except:
            print("curly bracket missing")
            sys.exit()


if len(round_brackets_stack) == 0 and len(curly_brackets_stack) == 0:
    print("your brackets are good")
else:
    print("check your brackets")
    sys.exit()


# - find the name of method
methodDecalration = re.findall(r'.*[public,private]?\s*[static]?\s*\w+\s*[\[]?\s*[\]]?\s+[a-zA-Z,\_,\$]{1}\w*\s*\(.*\)',allJavaCode)
if len(methodDecalration) > 1:
    print("please make sure there is only one method")
    sys.exit()
elif len (methodDecalration)==1:
    open_round_bracket_index = methodDecalration[0].index("(")
    for i in range(0,len(methodDecalration[0])):
        if methodDecalration[0][i] == " " and i < open_round_bracket_index-1:
            if " " not in methodDecalration[0][i:open_round_bracket_index].strip() :
                space_index = i
    method_name = methodDecalration[0][space_index:open_round_bracket_index].strip()

# - find the input of the method
if len (methodDecalration)==1:
    close_round_bracket_index = methodDecalration[0].index(")")
    inputs = methodDecalration[0][open_round_bracket_index+1 :close_round_bracket_index].strip() 
    raw_input_list = inputs.split(',')
    input_list = []
   
    for input in raw_input_list:
        input = input.strip()
        for i in range(0,len(input)):
            if input[i] == " ":
                if " " not in input[i+1:len(input)]:
                    space_index = i
        input_list.append(input[space_index:len(input)].strip())


# - create indentation
def indent(spaces):

    indented = ""
    for i in range(0,spaces*4):
        indented = indented + " "

    return indented

# - the convert everything inside the method

def convert(allJavaCode, spaces,method_content):
    method_start = allJavaCode.index("{")
    index = 1
    while allJavaCode[len(allJavaCode)-index] != "}":
        index = index + 1
        method_end = len(allJavaCode) - index
         
    raw_method_content = allJavaCode[method_start+1:method_end].strip()
    #inst_list = raw_method_content.split(';')
    inst_list = re.split(r'[;{]+',raw_method_content)
    print(inst_list)
    

    # - for condition helper vaiables
    forinit = 0
    forterm = 0
    forincr = 0
    fornum1 =0 
    fornum2 =0



    for inst in inst_list:
        inst =  inst.strip()


        # remove indentation        
        if re.match(r'\}',inst):
            index = 0
            spaces = spaces - len(re.findall(r'\}',inst))

            index = len(inst)-1
            while(inst[index] != '}'):
                index = index -1

            inst = inst[index+1:len(inst)-1].strip() 
            print(inst)

        # - for loop condition
        if re.split(r'\s\(',inst)[0] == "for" or forinit == 1 or forterm == 1 or forincr == 1 :
              
            if forinit == 0  and forincr == 0 and forterm == 0:
                forinit = 1

            if forincr == 1:
                forint = 0
                forincr = 0
                forterm = 0
                method_content.append(indent(spaces)+"Repeat "+ str(fornum2 - fornum1)+" times")
                spaces = spaces +1 
                
            if forterm == 1:
                forincr = 1
                forterm = 0
                forinit = 0
                if re.match(r'.*\<\s*\d+',inst):
                    fornum2 = int(re.split(r'\<',inst)[len(re.split(r'\<',inst))-1])
                if re.match(r'.*\<\=.*',inst):
                    fornum2 = int(re.split(r'\<\=',inst)[len(re.split(r'\<\=',inst))-1])+1


            if forinit == 1 :
                forterm = 1
                forinit = 0
                forincr = 0
                if re.match(r'.*\=.*',inst):
                    
                    fornum1 = 1 
                    #print re.split(r'\=',inst)[len(re.split(r'\=',inst))-1]
                

        # - system.out.print statments
        if "System.out.print" in inst :
            print_to_screen = re.findall(r'\".*\"',inst)
            
            print_to_screen[0] = "Print "+ print_to_screen[0]
            method_content.append(print_to_screen[0])




        # - int statments
        if re.split(r'[\s]',inst)[0] == "int" and "[]" not in inst :
            
            int_variable_name = re.split(r'\s',inst)[1]
            if "=" in inst:
                int_value = re.split(r'=',inst)[1].strip()
                method_content.append(indent(spaces)+int_variable_name+u" \u2190 "+int_value)
                
        # - char statement
        if re.split(r'\s',inst)[0] == "char" and "[]" not in inst :

            char_variable_name = re.split(r'\s',inst)[1]
            if "=" in inst:
                char_value = re.split(r'=',inst)[1].strip()
                method_content.append(indent(spaces)+char_variable_name+u" \u2190 "+char_value)





method_content = []
convert(allJavaCode,0,method_content)


        

print("----------------------------------------------------")
print("Algothrim:" + method_name)
print("----------------------------------------------------")
print("INPUT: " + ", ".join(input_list))
print("OUPUT: ")

print("----------------------------------------------------\n")
for inst in method_content:
    print(inst)


your brackets are good
please make sure there is only one method


SystemExit: 

In [10]:
'''
INSTRUCTIONS

1. Create a file with the following code
2. Put the file you want to convert into the same folder as it, and rename it to "file.py"
3. Add a "#F" comment to any lines in the code which have a function call that doesn't assign anything (so no =),
as the program cannot handle these convincingly
4. Run the converter file
'''

import re

python_file = r"C:\sunil\ipynb\file.py"
work_file = None

basic_conversion_rules = {"for": "FOR", "=": "TO", "if": "IF", "==": "EQUALS", "while": "WHILE", "until": "UNTIL", "import": "IMPORT", "class": "DEFINE CLASS", "def": "DEFINE FUNCTION", "else:": "ELSE:", "elif": "ELSEIF", "except:": "EXCEPT:", "try:": "TRY:", "pass": "PASS", "in": "IN"}
prefix_conversion_rules = {"=": "SET ", "#F": "CALL "}
advanced_conversion_rules = {"print": "OUTPUT", "return": "RETURN", "input": "INPUT"}

def f2list(to_list):
    return to_list.readlines()

def l2pseudo(to_pseudo):
    for line in to_pseudo:
        line_index = to_pseudo.index(line)
        line = str(line)
        line = re.split(r'(\s+)', line)
        for key, value in prefix_conversion_rules.items():
            if key in line:
                if not str(line[0]) == '':
                    line[0] = value + line[0]
                else:
                    line[2] = value + line[2]
        for key, value in basic_conversion_rules.items():
            for word in line:
                if key == str(word):
                    line[line.index(word)] = value
        for key, value in advanced_conversion_rules.items():
            for word in line:
                line[line.index(word)] = word.replace(key, value)
        for key, value in prefix_conversion_rules.items():
            for word in line:
                if word == key:
                    del line[line.index(word)]
        to_pseudo[line_index]= "".join(line)
    return(to_pseudo)

def p2file(to_file):
    file = open(python_file + '_pseudo.txt', 'w')
    for line in to_file:
        print(line, file=file)

def main():
    main_file = open(python_file, 'r+')
    work_file = f2list(main_file)
    work_file = l2pseudo(work_file)
    p2file(work_file)
    

main()